<a href="https://colab.research.google.com/github/unie12/recommendation/blob/main/cf_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import psutil


from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix, coo_matrix
from concurrent.futures import ProcessPoolExecutor


import warnings
warnings.filterwarnings("ignore")
from sklearn.utils import shuffle

import joblib

In [ ]:
# 데이터 로드 (청크 단위로 처리)
def load_data_in_chunks(file_path, chunk_size=100000):
    chunks = []
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        chunks.append(chunk)
    return pd.concat(chunks, ignore_index=True)

In [ ]:
ratings_df = shuffle(ratings_df, random_state=2021)
cutoff = int(TRAIN_SIZE * len(ratings_df))
ratings_train = ratings_df.iloc[:cutoff]
ratings_test = ratings_df.iloc[cutoff:]

In [ ]:
# 메모리 사용량 모니터링 함수
def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024  # MB 단위로 반환

In [ ]:
def load_model_data(model_dir='/content'):
    model_path = os.path.join(model_dir, 'cf_more_data.pkl')
    try:
        cf_model = joblib.load(model_path)
        print("Successfully loaded the model")

        # 로드 후 상태 출력
        print("Model attributes after loading:", cf_model.__dict__.keys())

        return cf_model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

In [ ]:
import os
import pandas as pd
import numpy as np
import psutil
import re

from tqdm import tqdm
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse import csr_matrix, coo_matrix
from concurrent.futures import ProcessPoolExecutor

import warnings
warnings.filterwarnings("ignore")
from sklearn.utils import shuffle

import joblib

# 데이터 로드 (청크 단위로 처리)
def load_data_in_chunks(file_path, chunk_size=100000):
    chunks = []
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        chunks.append(chunk)
    return pd.concat(chunks, ignore_index=True)

movies_df = load_data_in_chunks('/content/movie.csv')
ratings_df = load_data_in_chunks('/content/rating.csv')


# 사용자 ID 140000의 평점 데이터 추출
user_id_to_include = 140000
user_ratings = ratings_df[ratings_df['userId'] == user_id_to_include]


movies_df_filtered = movies_df
ratings_df_filtered = ratings_df[ratings_df['movieId'].isin(movies_df_filtered['movieId'])]
# ratings_df_filtered = pd.concat([ratings_df_filtered, user_ratings]).drop_duplicates()

# 사용자 ID 140000의 데이터가 포함되어 있는지 확인
print("사용자 ID 140000의 데이터:")
print(ratings_df_filtered[ratings_df_filtered['userId'] == 140000])

# 필터링 전과 후의 ratings 데이터 크기 출력
print(f"Original ratings size: {len(ratings_df)}")
print(f"Filtered ratings size: {len(ratings_df_filtered)}")

# 데이터 분할
TRAIN_SIZE = 0.75

ratings_df_filtered = shuffle(ratings_df_filtered, random_state=2021)
cutoff = int(TRAIN_SIZE * len(ratings_df_filtered))
ratings_train = ratings_df_filtered.iloc[:cutoff]
ratings_test = ratings_df_filtered.iloc[cutoff:]

print("User 140000 data in training set:")
print(ratings_train[ratings_train['userId'] == 140000])
print("\nUser 140000 data in test set:")
print(ratings_test[ratings_test['userId'] == 140000])

사용자 ID 140000의 데이터:
         userId  movieId  rating             timestamp
1949366  140000    79132     5.0  2024-10-12 12:00:00 
1949367  140000    56782     5.0   2024-10-12 12:00:00
1949368  140000    76091     5.0   2024-10-12 12:00:00
1949369  140000    61018     5.0   2024-10-12 12:00:00
1949370  140000      123     3.5   2024-10-12 12:00:00
1949371  140000     5989     4.5   2024-10-12 12:00:00
1949372  140000    58559     5.0   2024-10-12 12:00:00
1949373  140000      296     5.0  2024-10-12 12:00:00 
1949374  140000      306     4.5  2024-10-12 12:00:00 
1949375  140000      307     4.0  2024-10-12 12:00:00 
1949376  140000      308     4.0  2024-10-12 12:00:00 
1949377  140000       16     3.5  2024-10-12 12:00:00 
1949378  140000      111     4.0  2024-10-12 12:00:00 
1949379  140000     1227     4.0  2024-10-12 12:00:00 
1949380  140000      858     4.0  2024-10-12 12:00:00 
1949381  140000    48394     3.0  2024-10-12 12:00:00 
1949382  140000   111622     4.0  2024-10-12 

In [ ]:
# 메모리 사용량 모니터링 함수
def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024  # MB 단위로 반환

class CollaborativeFiltering:
    def __init__(self, ratings_df_filtered, hyper_params):
        self.R = ratings_df_filtered
        self.num_users = len(np.unique(self.R.row))
        self.num_items = len(np.unique(self.R.col))
        self.K = hyper_params['K']
        self.alpha = hyper_params['alpha']
        self.beta = hyper_params['beta']
        self.iterations = hyper_params['iterations']
        self.verbose = hyper_params['verbose']
        self.batch_size = hyper_params['batch_size']
        self.b = np.mean(self.R.data)  # 전체 평균 rating 초기화

        self._create_mappings()
        self.initialize_factors()

    def _create_mappings(self):
        self.user_id_map = {id: idx for idx, id in enumerate(np.unique(self.R.row))}
        self.item_id_map = {id: idx for idx, id in enumerate(np.unique(self.R.col))}
        self.idx_user_map = {v: k for k, v in self.user_id_map.items()}
        self.idx_item_map = {v: k for k, v in self.item_id_map.items()}

    def initialize_factors(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K)).astype(np.float32)
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K)).astype(np.float32)
        self.b_u = np.zeros(self.num_users, dtype=np.float32)
        self.b_d = np.zeros(self.num_items, dtype=np.float32)

    def rmse(self):
        xs, ys = self.R.nonzero()
        predictions = self.get_prediction(xs, ys)
        errors = self.R.data - predictions.flatten()
        return np.sqrt(np.mean(errors**2))

    def get_prediction(self, i, j):
        return self.b + self.b_u[i] + self.b_d[j] + np.sum(self.P[i] * self.Q[j], axis=1)

    def sgd_batch(self, batch):
        i, j, r = zip(*batch)
        i, j, r = np.array(i, dtype=int), np.array(j, dtype=int), np.array(r)
        prediction = self.get_prediction(i, j)
        e = r - prediction

        self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
        self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

        P_i = self.P[i]
        Q_j = self.Q[j]

        e_reshaped = e.reshape(-1, 1)
        self.P[i] += self.alpha * (e_reshaped * Q_j - self.beta * P_i)
        self.Q[j] += self.alpha * (e_reshaped * P_i - self.beta * Q_j)

    def set_test(self, ratings_test):
        test_set = []
        for _, row in ratings_test.iterrows():
            user = row['userId']
            item = row['movieId']
            if user in self.user_id_map and item in self.item_id_map:
                i, j = self.user_id_map[user], self.item_id_map[item]
                test_set.append((i, j, row['rating']))

        self.test_set = test_set
        return test_set

    def test_rmse(self):
        test_set = np.array(self.test_set)
        predictions = self.get_prediction(test_set[:, 0].astype(int), test_set[:, 1].astype(int))
        errors = test_set[:, 2] - predictions
        return np.sqrt(np.mean(errors**2))

    def train(self):
        samples = list(zip(self.R.nonzero()[0], self.R.nonzero()[1], self.R.data))

        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(samples)

            for j in range(0, len(samples), self.batch_size):
                batch = samples[j:j+self.batch_size]
                try:
                    self.sgd_batch(batch)
                except MemoryError:
                    print("MemoryError occurred during training. Trying to reduce memory usage...")
                    # 배치 크기 줄이기
                    self.batch_size = max(1, self.batch_size // 2)
                    continue  # 현재 배치에서 계속 진행

                # 중간 결과 삭제
                del batch

            if self.verbose and (i+1) % 10 == 0:
                rmse1 = self.rmse()
                rmse2 = self.test_rmse()
                training_process.append((i+1, rmse1, rmse2))
                print(f'Iteration : {i+1} ; Train RMSE = {rmse1:.4f} ; Test RMSE = {rmse2:.4f}')

        return training_process

    def predict(self, user_id, item_id):
        if user_id in self.user_id_map and item_id in self.item_id_map:
            return self.get_prediction(self.user_id_map[user_id], self.item_id_map[item_id])[0]
        else:
            return self.b  # 평균 평점 반환

    def full_prediction(self):
        return self.b + self.b_u[:, np.newaxis] + self.b_d[np.newaxis, :] + self.P.dot(self.Q.T)

    def predict_for_user(self, user_id):
        if user_id in self.user_id_map:
            user_idx = self.user_id_map[user_id]
            return self.b + self.b_u[user_idx] + self.b_d + np.dot(self.P[user_idx], self.Q.T)
        else:
            return np.full(self.num_items, self.b)  # 평균 평점으로 채운 배열 반환

    def __getstate__(self):
        state = self.__dict__.copy()
        if 'R' in state:
            del state['R']
        return state

    def __setstate__(self, state):
        self.__dict__.update(state)
        self.R = None

In [ ]:
# 데이터 준비
user_ids = ratings_df_filtered['userId'].unique()
movie_ids = ratings_df_filtered['movieId'].unique()

# 데이터 준비
user_ids = ratings_df_filtered['userId'].unique()
movie_ids = ratings_df_filtered['movieId'].unique()

# 사용자 ID 140000 ratings_df_filtered에 포함되어 있는지 확인
if 140000 not in user_ids:
    print("User ID 140000 is not in ratings_df_filtered. Please check the filtering process.")
else:
    user_map = {user_id: index for index, user_id in enumerate(user_ids)}
    movie_map = {movie_id: index for index, movie_id in enumerate(movie_ids)}

    row_indices = ratings_df_filtered['userId'].map(user_map)
    col_indices = ratings_df_filtered['movieId'].map(movie_map)
    data = ratings_df_filtered['rating']

    # coo_matrix 생성
    sparse_mat = coo_matrix((data, (row_indices, col_indices)), shape=(len(user_ids), len(movie_ids)))

    # user_id_map 확인
    print("User ID 140000 in user_id_map:", 140000 in user_map)  # user_map에서 확인
    print("User ID 140000 mapped index:", user_map.get(140000, "Not found"))  # user_map에서 인덱스 확인

# 하이퍼파라미터 설정
hyper_params = {
    'K': 50,
    'alpha': 0.001,
    'beta': 0.02,
    'iterations': 50,
    'verbose': True,
    'batch_size': 512,
    'early_stopping_rounds': 5,
    'n_jobs': -1
}

User ID 140000 in user_id_map: True
User ID 140000 mapped index: 1349


In [ ]:
import pandas as pd
import re

# 연도 추출 함수
def extract_year(title):
    match = re.search(r'\((\d{4})\)', title)
    if match:
        return int(match.group(1))
    return None

# 데이터 로드
ratings_df = pd.read_csv('rating.csv')
movies_df = pd.read_csv('movie.csv')

# 연도 열 추가
movies_df['year'] = movies_df['title'].apply(extract_year)

# ratings_df와 movies_df 병합
merged_df = pd.merge(ratings_df, movies_df, on='movieId')

# 연도별 평점 수 집계
yearly_ratings_count = merged_df['year'].value_counts().sort_index()

# 오름차순으로 정렬된 연도별 평점 수 출력
sorted_yearly_ratings_count = yearly_ratings_count.sort_values()
print("Yearly Ratings Count (Sorted in Ascending Order):")
print(sorted_yearly_ratings_count.tail(50))

Yearly Ratings Count (Sorted in Ascending Order):
year
1954.0      6180
1959.0      6368
1963.0      6556
2013.0      6905
1962.0      7071
1967.0      7952
1964.0      8550
1972.0      8591
1976.0      9438
2012.0      9515
1974.0      9994
1968.0     10172
1973.0     10627
1978.0     11079
1971.0     12316
2011.0     12320
1977.0     12434
1975.0     14308
2010.0     17150
1979.0     17369
1983.0     19123
1981.0     19202
1980.0     20926
2009.0     22217
1982.0     24180
2008.0     26478
1985.0     27671
2007.0     31420
1984.0     32761
1987.0     34252
1988.0     34448
1986.0     34992
2005.0     35053
2006.0     35272
1991.0     38805
1989.0     45007
1990.0     45315
1992.0     46299
2003.0     50180
2004.0     52674
2002.0     59974
2001.0     66864
2000.0     80153
1998.0     81622
1997.0     84595
1993.0     91218
1999.0    101123
1996.0    110809
1994.0    127022
1995.0    148289
Name: count, dtype: int64


In [ ]:
# 모델 학습
cf_model = CollaborativeFiltering(sparse_mat, hyper_params)

print("Setting up test set...")
test_set = cf_model.set_test(ratings_test)
print("Test set setup complete.")

print(f"Initial memory usage: {get_memory_usage():.2f} MB")

def train_with_memory_management():
    global cf_model, hyper_params
    try:
        result = cf_model.train()
    except MemoryError:
        print("MemoryError occurred during training. Trying to reduce memory usage...")
        del cf_model
        hyper_params['K'] = max(5, hyper_params['K'] // 2)
        hyper_params['batch_size'] *= 2
        cf_model = CollaborativeFiltering(sparse_mat, hyper_params)
        test_set = cf_model.set_test(ratings_test)
        try:
            result = cf_model.train()
        except MemoryError:
            print("MemoryError still occurring. Consider using a machine with more memory or further optimizing the code.")
            return None
    return result

result = train_with_memory_management()


Setting up test set...
Test set setup complete.
Initial memory usage: 923.14 MB
Iteration : 10 ; Train RMSE = 0.8698 ; Test RMSE = 1.2083
Iteration : 20 ; Train RMSE = 0.8587 ; Test RMSE = 1.2405
Iteration : 30 ; Train RMSE = 0.8538 ; Test RMSE = 1.2556
Iteration : 40 ; Train RMSE = 0.8490 ; Test RMSE = 1.2641
Iteration : 50 ; Train RMSE = 0.8392 ; Test RMSE = 1.2689


In [ ]:
if result is not None:
    def save_model_data(cf_model, user_map, movie_map, movies_df, ratings_df, model_dir='/content/model_data'):
      os.makedirs(model_dir, exist_ok=True)
      model_path = os.path.join(model_dir, 'cf_model_v2.pkl')
      data_to_save = {
          'model': cf_model,
          'user_map': user_map,
          'movie_map': movie_map,
          'movies_df': movies_df_filtered,
          'ratings_df': ratings_df_filtered  # ratings_df 추가

      }
      joblib.dump(data_to_save, model_path)
      print(f"Model and data saved in {model_path}")

    # 모델 저장
    save_model_data(cf_model, user_map, movie_map, movies_df_filtered, ratings_df_filtered)

    print(f"Final memory usage: {get_memory_usage():.2f} MB")

    # 모델 평가 및 분석
    print(f"movies_df의 영화 수: {len(movies_df_filtered)}")
    print(f"모델의 영화 수: {len(cf_model.item_id_map)}")

    model_movie_ids = set(cf_model.item_id_map.keys())
    df_movie_ids = set(movies_df_filtered['movieId'])

    print(f"공통 영화 ID 수: {len(model_movie_ids & df_movie_ids)}")
    print(f"movies_df에만 있는 영화 ID 수: {len(df_movie_ids - model_movie_ids)}")
    print(f"모델에만 있는 영화 ID 수: {len(model_movie_ids - df_movie_ids)}")

    mismatched_ids = list(df_movie_ids - model_movie_ids)[:5]
    print("\n불일치하는 영화 ID 예시:")
    for movie_id in mismatched_ids:
        movie = movies_df_filtered[movies_df_filtered['movieId'] == movie_id].iloc[0]
        print(f"Movie ID: {movie_id}, Title: {movie['title']}")

    # # 사용자별 추천 영화 출력
    # for user_id in [140000]:
    #     user_predictions = cf_model.predict_for_user(user_id)
    #     top_movie_indices = np.argsort(user_predictions)[::-1][:15]
    #     top_movie_ids = [cf_model.idx_item_map[idx] for idx in top_movie_indices]
    #     recommended_movies = movies_df_filtered[movies_df_filtered['movieId'].isin(top_movie_ids)]

    #     print(f"\n사용자 {user_id}에 대한 추천 영화:")
    #     print(f"예측된 영화 수: {len(user_predictions)}")
    #     print(f"최소 예측 평점: {np.min(user_predictions):.2f}")
    #     print(f"최대 예측 평점: {np.max(user_predictions):.2f}")

    #     print("\n상위 10개 추천 영화:")
    #     for idx, movie in recommended_movies.iterrows():
    #         movie_id = movie['movieId']
    #         predicted_rating = user_predictions[cf_model.item_id_map[movie_id]]
    #         print(f"제목: {movie['title']}, 장르: {movie['genres']}, 예측 평점: {predicted_rating:.2f}")

    # 모델 성능 평가
    train_rmse = cf_model.rmse()
    test_rmse = cf_model.test_rmse()
    print(f"\n최종 Train RMSE: {train_rmse:.4f}")
    print(f"최종 Test RMSE: {test_rmse:.4f}")
else:
    print("모델 학습에 실패했습니다. 메모리 문제를 해결하지 못했습니다.")


Model and data saved in /content/model_data/cf_model_v2.pkl
Final memory usage: 1206.97 MB
movies_df의 영화 수: 27280
모델의 영화 수: 19100
공통 영화 ID 수: 8337
movies_df에만 있는 영화 ID 수: 18943
모델에만 있는 영화 ID 수: 10763

불일치하는 영화 ID 예시:
Movie ID: 131072, Title: Jesus liebt mich (2012)
Movie ID: 98304, Title: So Big! (1932)
Movie ID: 131074, Title: Mount St. Elias (2009)
Movie ID: 32770, Title: Brothers (Brødre) (2004)
Movie ID: 131076, Title: Süperseks (2004)

최종 Train RMSE: 0.8392
최종 Test RMSE: 1.2689


In [ ]:
user_index = user_map[140000]

# 사용자별 추천 영화 출력
user_predictions = cf_model.predict_for_user(user_index)
top_movie_indices = np.argsort(user_predictions)[::-1][:25]
top_movie_ids = [cf_model.idx_item_map[idx] for idx in top_movie_indices]
recommended_movies = movies_df_filtered[movies_df_filtered['movieId'].isin(top_movie_ids)]

print(f"\n사용자 {140000}에 대한 추천 영화:")
print(f"예측된 영화 수: {len(user_predictions)}")
print(f"최소 예측 평점: {np.min(user_predictions):.2f}")
print(f"최대 예측 평점: {np.max(user_predictions):.2f}")

print("\n상위 10개 추천 영화:")
for idx, movie in recommended_movies.iterrows():
    movie_id = movie['movieId']
    predicted_rating = user_predictions[cf_model.item_id_map[movie_id]]
    print(f"제목: {movie['title']}, 장르: {movie['genres']}, 예측 평점: {predicted_rating:.2f}")


사용자 140000에 대한 추천 영화:
예측된 영화 수: 19100
최소 예측 평점: 1.57
최대 예측 평점: 4.59

상위 10개 추천 영화:
제목: Four Rooms (1995), 장르: Comedy, 예측 평점: 4.48
제목: Babe (1995), 장르: Children|Drama, 예측 평점: 4.48
제목: Browning Version, The (1994), 장르: Drama, 예측 평점: 4.48
제목: Song of the Little Road (Pather Panchali) (1955), 장르: Drama, 예측 평점: 4.48
제목: Low Life (1994), 장르: Drama, 예측 평점: 4.54
제목: Arrival, The (1996), 장르: Action|Sci-Fi|Thriller, 예측 평점: 4.59
제목: Crow: City of Angels, The (1996), 장르: Action|Thriller, 예측 평점: 4.53
제목: Ben-Hur (1959), 장르: Action|Adventure|Drama, 예측 평점: 4.51
제목: Nenette and Boni (Nénette et Boni) (1996), 장르: Drama, 예측 평점: 4.47
제목: Twisted (1996), 장르: Comedy|Drama, 예측 평점: 4.47
제목: Henry Fool (1997), 장르: Comedy|Drama, 예측 평점: 4.48
제목: Shaggy D.A., The (1976), 장르: Children|Comedy, 예측 평점: 4.47
제목: Suburbans, The (1999), 장르: Drama, 예측 평점: 4.45
제목: Kelly's Heroes (1970), 장르: Action|Comedy|War, 예측 평점: 4.49
제목: Time Machine, The (1960), 장르: Action|Adventure|Sci-Fi, 예측 평점: 4.48
제목: Magnificent Seven, The (

In [ ]:
# 사용자 ID 140000이 user_id_map에 매핑된 인덱스 확인
user_id = 140000
if user_id in user_map:
    mapped_index = user_map[user_id]
    print(f"User ID {user_id} is mapped to index {mapped_index} in user_map.")
else:
    print(f"User ID {user_id} is not found in user_map.")

# COO Matrix의 row_indices에서 사용자 ID 140000의 인덱스 확인
row_indices = ratings_df_filtered['userId'].map(user_map)
print(f"Row index for User ID {user_id}: {row_indices[ratings_df_filtered['userId'] == user_id].unique()}")

User ID 140000 is mapped to index 1349 in user_map.
Row index for User ID 140000: [1349]
